# PowerShell Parameter Binding
What: Things that dictate parameter binding in cmdlets and advanced functions
- Does the given parameter take pipeline input by either value or property name?
- The _Type_ of either pipeline object or property of object for ByPropertyName
- can pipeline object/property be cast ("coerced") into the type of the parameter?
- `ParameterSet` -- what `ParameterSet` is in play, and which parameters therein are primed to accept value from pipeline?

Here we will observe how PowerShell handles Parameter Binding through some examples with a couple of cmdlets, and of a couple of variations of a script. Will will nspect the parameters, and then trace how the binding happens:
- `Get-Service`
- `Import-Module`
- `.\Test-ValueFromPipeline.ps1`

## Examples
A few examples to see where things get bound

### `Get-Service` Example

First, let's look at the parameters of the cmdlet `Get-Service`.

In [31]:
Get-Help -Parameter * -Name Get-Service | Format-Table -Property name, pipelineInput, position, @{n="type"; e={$_.Type.Name}}


name              pipelineInput                  position type
----              -------------                  -------- ----
DependentServices False                          named    System.Management.Automation.SwitchParam…
DisplayName       False                          named    System.String[]
Exclude           False                          named    System.String[]
Include           False                          named    System.String[]
InputObject       True (ByValue)                 named    System.ServiceProcess.ServiceController[]
Name              True (ByPropertyName, ByValue) 0        System.String[]
RequiredServices  False                          named    System.Management.Automation.SwitchParam…



We see that two of the parameters (`InputObject`, `-Name`) take pipeline input `ByValue` (the whole object), and one parameter (`-Name`) takes pipeline input `ByPropertyName` (if the pipeline object has a property named `Name`).

So, let's see what the behavior is when we pipe a `String` object to the `Get-Service` cmdlet, and determine the parameter to which this object is bound. Aside: Here we're using the `Trace-Command` cmdlet (from the stock module `Microsoft.PowerShell.Utility` that comes with PowerShell). This cmdlet has a multitude of uses, just one of which is to trace the ParameterBinding aspects of an expression

In [32]:
Trace-Command -Expression {"WebClient" | Get-Service} -Name ParameterBinding, ParameterBinderController -PSHost

DEBUG: 2021-09-13 17:21:44.6876 ParameterBinderController Information: 0 :  WriteLine   Argument count: 0
DEBUG: 2021-09-13 17:21:44.6890 ParameterBinding Information: 0 : BIND NAMED cmd line args [Get-Service]
DEBUG: 2021-09-13 17:21:44.6895 ParameterBinding Information: 0 : BIND POSITIONAL cmd line args [Get-Service]
DEBUG: 2021-09-13 17:21:44.6897 ParameterBinderController Information: 0 :  WriteLine   CurrentParameterSetName = Default
DEBUG: 2021-09-13 17:21:44.6899 ParameterBinderController Information: 0 :  WriteLine   CurrentParameterSetName = Default
DEBUG: 2021-09-13 17:21:44.6900 ParameterBinding Information: 0 : MANDATORY PARAMETER CHECK on cmdlet [Get-Service]
DEBUG: 2021-09-13 17:21:44.6902 ParameterBinding Information: 0 : CALLING BeginProcessing
DEBUG: 2021-09-13 17:21:44.6903 ParameterBinding Information: 0 : BIND PIPELINE object to parameters: [Get-Service]
DEBUG: 2021-09-13 17:21:44.6905 ParameterBinding Information: 0 :     PIPELINE object TYPE = [System.String]
DEBU

Among all of that useful debugging information, we see the `SUCCESSFUL` result for the attempt to bind the pipeline object (of type `String`) to `Parameter [Name]`. Cool!

Though, the question might arise, "why did PowerShell bind to `-Name` and not to `-InputObject`, the other parameter that accepts pipeline input `ByValue`?". Good question. PowerShell uses an object's Type in the binding process, too. Let's see this in action for another example with `Get-Service`, this time with a `ServiceController` object from the pipeline 

In [33]:
Trace-Command -Name ParameterBinding, ParameterBinderController -PSHost -Expression {[System.ServiceProcess.ServiceController]::new("WebClient") | Get-Service}

DEBUG: 2021-09-13 17:21:48.7162 ParameterBinderController Information: 0 :  WriteLine   Argument count: 0
DEBUG: 2021-09-13 17:21:48.7165 ParameterBinding Information: 0 : BIND NAMED cmd line args [Get-Service]
DEBUG: 2021-09-13 17:21:48.7167 ParameterBinding Information: 0 : BIND POSITIONAL cmd line args [Get-Service]
DEBUG: 2021-09-13 17:21:48.7169 ParameterBinderController Information: 0 :  WriteLine   CurrentParameterSetName = Default
DEBUG: 2021-09-13 17:21:48.7172 ParameterBinderController Information: 0 :  WriteLine   CurrentParameterSetName = Default
DEBUG: 2021-09-13 17:21:48.7174 ParameterBinding Information: 0 : MANDATORY PARAMETER CHECK on cmdlet [Get-Service]
DEBUG: 2021-09-13 17:21:48.7177 ParameterBinding Information: 0 : CALLING BeginProcessing
DEBUG: 2021-09-13 17:21:48.7179 ParameterBinding Information: 0 : BIND PIPELINE object to parameters: [Get-Service]
DEBUG: 2021-09-13 17:21:48.7182 ParameterBinding Information: 0 :     PIPELINE object TYPE = [System.ServiceProce

Ah-ha! This time PowerShell bound the pipeline object (of type `ServiceController`) to the `-InputObject` parameter successfully. An exmaple of one of the many ways that PowerShell optimizes us, the consumers -- we can pipe things to a cmdlet in several ways, making it most flexible for the variety of situations in which we eventually find ourselves!

### `Import-Module` example

Alright, another similar example, this time with the `Import-Module` cmdlet, which has several ParameterSets. We should see that the parameter binding considers the pipeline object type and then binds the object value to the corresponding parameter. First, the parameter information for the cmdlet:

In [34]:
Get-Help -Parameter * -Name Import-Module | Format-Table -Property name, pipelineInput, position, type


name                 pipelineInput  position type
----                 -------------  -------- ----
Alias                False          named    @{name=System.String[]; uri=}
ArgumentList         False          named    @{name=System.Object[]; uri=}
AsCustomObject       False          named    @{name=System.Management.Automation.SwitchParameter; …
Assembly             True (ByValue) 0        @{name=System.Reflection.Assembly[]; uri=}
CimNamespace         False          named    @{name=System.String; uri=}
CimResourceUri       False          named    @{name=System.Uri; uri=}
CimSession           False          named    @{name=Microsoft.Management.Infrastructure.CimSession…
Cmdlet               False          named    @{name=System.String[]; uri=}
DisableNameChecking  False          named    @{name=System.Management.Automation.SwitchParameter; …
Force                False          named    @{name=System.Management.Automation.SwitchParameter; …
FullyQualifiedName   True (ByValue) 0      

So, `-Assembly`, `-FullyQualifiedName`, `-ModuleInfo`, and `-Name`.  Let's see how parameter binding behaves when we pipe in a string object:

In [2]:
Trace-Command -Name ParameterBinding, ParameterBinderController -PSHost -Expression {"posh-git" | Import-Module}

DEBUG: 2021-09-13 17:22:44.8065 ParameterBinderController Information: 0 :  WriteLine   Argument count: 0
DEBUG: 2021-09-13 17:22:44.8136 ParameterBinding Information: 0 : BIND NAMED cmd line args [Import-Module]
DEBUG: 2021-09-13 17:22:44.8143 ParameterBinding Information: 0 : BIND POSITIONAL cmd line args [Import-Module]
DEBUG: 2021-09-13 17:22:44.8147 ParameterBinderController Information: 0 :  WriteLine   CurrentParameterSetName = Name
DEBUG: 2021-09-13 17:22:44.8150 ParameterBinderController Information: 0 :  WriteLine   CurrentParameterSetName = Name
DEBUG: 2021-09-13 17:22:44.8153 ParameterBinding Information: 0 : MANDATORY PARAMETER CHECK on cmdlet [Import-Module]
DEBUG: 2021-09-13 17:22:44.8156 ParameterBinding Information: 0 : CALLING BeginProcessing
DEBUG: 2021-09-13 17:22:44.8159 ParameterBinding Information: 0 : BIND PIPELINE object to parameters: [Import-Module]
DEBUG: 2021-09-13 17:22:44.8162 ParameterBinding Information: 0 :     PIPELINE object TYPE = [System.String]
DE

Alright, as expected -- we piped in a String object, and PowerShell bound that string to the parameter that accepts pipeline input by value, and expects a String object. Great.

How about if we try piping in a ModuleSpecification object? We'd surely see PowerShell bind that object to the parameter that accepts pipeline input by value, and expects a ModuleSpecification object, right? Let's find out:

In [3]:
Trace-Command -Name ParameterBinding, ParameterBinderController -PSHost -Expression {[Microsoft.PowerShell.Commands.ModuleSpecification]::new("posh-git") | Import-Module}

DEBUG: 2021-09-13 17:22:48.6086 ParameterBinderController Information: 0 :  WriteLine   Argument count: 0
DEBUG: 2021-09-13 17:22:48.6090 ParameterBinding Information: 0 : BIND NAMED cmd line args [Import-Module]
DEBUG: 2021-09-13 17:22:48.6092 ParameterBinding Information: 0 : BIND POSITIONAL cmd line args [Import-Module]
DEBUG: 2021-09-13 17:22:48.6094 ParameterBinderController Information: 0 :  WriteLine   CurrentParameterSetName = Name
DEBUG: 2021-09-13 17:22:48.6096 ParameterBinderController Information: 0 :  WriteLine   CurrentParameterSetName = Name
DEBUG: 2021-09-13 17:22:48.6098 ParameterBinding Information: 0 : MANDATORY PARAMETER CHECK on cmdlet [Import-Module]
DEBUG: 2021-09-13 17:22:48.6100 ParameterBinding Information: 0 : CALLING BeginProcessing
DEBUG: 2021-09-13 17:22:48.6102 ParameterBinding Information: 0 : BIND PIPELINE object to parameters: [Import-Module]
DEBUG: 2021-09-13 17:22:48.6105 ParameterBinding Information: 0 :     PIPELINE object TYPE = [Microsoft.PowerSh

Nailed it! We see PowerShell try to bind to a few other parameters, first (`-Name`, then `-ModuleInfo`, then `-Assembly`), none of which expect the type that we piped. The, we see success when binding to `-FullyQualifiedName` -- correct type, no coercion needed -- successful! We also notice that this parameter is a part of the ParameterSet of the same name, `FullyQualifiedName`. We can explore ParameterSets in another notebook.

### Parameters / ParameterSets Example
Now for a few examples to reinforce the learnings we had above. Here we'll use a couple of minimal scripts to further explore parameter binding. The scripts take some parameters and just return the parameters that were bound. 

Aside: these minimal scripts are referred to as "advanced scripts" in PowerShell documentation, as they use the `CmdletBinding` attribute so as to be able to employ many cmdlet-like features -- you can read more about this attribute at via the PowerShell help (`Get-Help -Name about_Functions_CmdletBindingAttribute`), or online at the [CmdletBinding documentation page](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_functions_cmdletbindingattribute)

First we'll have a look at the parameters for this script:

In [4]:
Get-Help -Parameter * -Name .\Test-ValueFromPipelineBehavior.ps1 | Format-Table -Property name, pipelineInput, position, type


name  pipelineInput  position type
----  -------------  -------- ----
Name  true (ByValue) 1        @{name=String[]}
Count true (ByValue) 2        @{name=Int32}
Start true (ByValue) 3        @{name=DateTime}



Three parameters, all of which accept pipeline input by value, and each of which expects different input object types. As we saw above, PowerShell will bind parameters based on the input object type, and may do some coercing to try to "fit" pipeline objects into values for parameters.

So, if we pipe in a String, we might expect PowerShell to bind that to the parameter `-Name`. Let's see:

In [5]:
"Dickie" | .\Test-ValueFromPipelineBehavior.ps1


Key  Value
---  -----
Name {Dickie}



Just as expected: the String `Dickie` was bound to the only parameter that expects a string, `-Name`. Let's see about an Int and a DateTime:

In [6]:
 Get-Date | .\Test-ValueFromPipelineBehavior.ps1


Key   Value
---   -----
Start 9/13/2021 5:23:04 PM
Name  {09/13/2021 17:23:04}



In [7]:
 2 | .\Test-ValueFromPipelineBehavior.ps1


Key                  Value
---                  -----
Count                    2
Start 1/1/0001 12:00:00 AM
Name                   {2}



What the world?! Why did a DateTime get bound to multiple parameters (parameters expecting a String and a DateTime, respectively), as did an Int?!  That's where the coercion comes in -- the input objects' types can be cast or coerced into the expected types for more than one of the script's parameters, and so they are!

Enter ParameterSets:
> PowerShell uses parameter sets to enable you to write a single function that can do different actions for different scenarios
- [about_Parameter_Sets help](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_parameter_sets), or `Get-Help -Name about_Parameter_Sets`

Let's have a look at the ParameterSet of which each parameter is a part for this script (the following code block, while not specific to parameter binding, gives us a wonderful example of the discoverability of PowerShell itself -- we can use PowerShell to learn more about PowerShell, its cmdlets, its parameters, etc!):

In [8]:
Get-Command .\Test-ValueFromPipelineBehavior.ps1 | Foreach-Object {$_.ParameterSets} -PipelineVariable oThisParamSet | Foreach-Object {
    $_.parameters | Where-Object {$_.Name -NotIn ([System.Management.Automation.Internal.CommonParameters].GetProperties()).Name} | Sort-Object -Property Name |
    Select-Object -Property Name, ParameterType, is*,
        @{n="Position"; e={if ($_.Position -lt 0) {"Named"} else {$_.Position}}},
        @{n="Alias"; e={$_.Aliases}},
        @{n="ParameterSet"; e={$oThisParamSet.Name}},
        @{n="IsDefaultParameterSet"; e={$oThisParamSet.IsDefault}}
} | Format-Table -AutoSize -GroupBy @{n="ParameterSet"; e={"{0}{1}" -f $_.ParameterSet, $(if ($_.IsDefaultParameterSet) {" (default)"})}}


   ParameterSet: __AllParameterSets

Name  ParameterType   IsMandatory IsDynamic Position Alias ParameterSet       IsDefaultParameterSet
----  -------------   ----------- --------- -------- ----- ------------       ---------------------
Count System.Int32          False     False        1       __AllParameterSets                 False
Name  System.String[]       False     False        0       __AllParameterSets                 False
Start System.DateTime       False     False        2       __AllParameterSets                 False



Three parameters, and a single parameter set (with the default name `__AllParameterSets`), got it.

Now let's have a look at similar examples, but using a slightly updated version of the script that employs parameter sets so as to have a more desirable/expected experience for the consumer:

In [9]:
Get-Help -Parameter * -Name .\Test-ValueFromPipelineBehavior_withParamSet.ps1 | Format-Table -Property name, pipelineInput, position, type


name  pipelineInput  position type
----  -------------  -------- ----
Name  true (ByValue) named    @{name=String[]}
Count true (ByValue) named    @{name=Int32}
Start true (ByValue) named    @{name=DateTime}



And, to point out that each parameter is in a different ParameterSet from the others, let's look at the command's `ParameterSets` info:

In [10]:
Get-Command .\Test-ValueFromPipelineBehavior_withParamSet.ps1 | Foreach-Object {$_.ParameterSets} -PipelineVariable oThisParamSet | Foreach-Object {
    $_.parameters | Where-Object {$_.Name -NotIn ([System.Management.Automation.Internal.CommonParameters].GetProperties()).Name} | Sort-Object -Property Name |
    Select-Object -Property Name, ParameterType, is*,
        @{n="Position"; e={if ($_.Position -lt 0) {"Named"} else {$_.Position}}},
        @{n="Alias"; e={$_.Aliases}},
        @{n="ParameterSet"; e={$oThisParamSet.Name}},
        @{n="IsDefaultParameterSet"; e={$oThisParamSet.IsDefault}}
} | Format-Table -AutoSize -GroupBy @{n="ParameterSet"; e={"{0}{1}" -f $_.ParameterSet, $(if ($_.IsDefaultParameterSet) {" (default)"})}}


   ParameterSet: ByName (default)

Name ParameterType   IsMandatory IsDynamic Position Alias ParameterSet IsDefaultParameterSet
---- -------------   ----------- --------- -------- ----- ------------ ---------------------
Name System.String[]       False     False Named          ByName                        True

   ParameterSet: ByCount

Name  ParameterType IsMandatory IsDynamic Position Alias ParameterSet IsDefaultParameterSet
----  ------------- ----------- --------- -------- ----- ------------ ---------------------
Count System.Int32        False     False Named          ByCount                      False

   ParameterSet: ByStart

Name  ParameterType   IsMandatory IsDynamic Position Alias ParameterSet IsDefaultParameterSet
----  -------------   ----------- --------- -------- ----- ------------ ---------------------
Start System.DateTime       False     False Named          ByStart                      False



Mhmmm -- three separate parameter sets! Now PowerShell should only bind a pipeline value to at most one parameter for this particular script (PowerShell uses just one parameter set at a time). Let's see:

In [11]:
"Dickie" | .\Test-ValueFromPipelineBehavior_withParamSet.ps1


Key  Value
---  -----
Name {Dickie}



In [12]:
2 | .\Test-ValueFromPipelineBehavior_withParamSet.ps1


Key   Value
---   -----
Count     2



In [13]:
Get-Date | .\Test-ValueFromPipelineBehavior_withParamSet.ps1


Key   Value
---   -----
Start 9/13/2021 5:23:23 PM



That's the stuff! So, ParameterSets also come in to play. We can use the `Get-Command` code blocks above to inspect parameter sets for any command/script (or any other command inspection/discovery technique you like), and we can further our understanding of how and why PowerShell binds parameters from pipeline the ways that it does.

And, now we know that PowerShell binds parameters based on parameter attributes (if the parameter accepts values from pipeline), on the object types that are coming through the pipeline, and the parameter sets defined in the cmdlet / script / function. Neat!